<img src="images/logo_simple.png" alt="logo" width="400"/>

# Snippets de code 

Ces snippets de code vont vous permettre de gagner du temps dans la prise en main des ressources à disposition et de l'écosystème data d'île-de-France Mobilités. 

### Appel d'API PRIM : exemple de demande de calcul d'itinéraire

In [ ]:
from requests.auth import HTTPBasicAuth
import requests
import json
import pandas as pd

#Affectation des coordonnées au départ et à l'arrivée
dlong = "2.33792"
dlat = "48.85827"
along = "2.3588523"
alat = "48.9271087"

#Date et heure du trajet
jour = "20241121T073000"

#URL de l'API 
destination = dlong + "%3B%20" + dlat + "&to=" + along + "%3B%20" + alat + "&datetime=" + jour
url = 'https://prim.iledefrance-mobilites.fr/marketplace/v2/navitia/journeys?from=' + destination

#Le header doit contenir la clé API : apikey, remplacer #VOTRE CLE API par votre clé API
headers = {'Accept': 'application/json', 'apikey': '#VOTRE CLE API'}

#Envoi de la requête au serveur
req = requests.get(url, headers=headers)

#Affichage du code réponse
print('Status:',req)

#Lecture du json
data = pd.json_normalize(req.json())

#Les différents trajets retournés sont dans data['journeys'][0]

### Appel d'API PRIM : passage du token et appel d'un endpoint

In [ ]:
# imports
import requests

# Clé d'API pour IDFM - Obtention possible via le portail PRIM
api_idfm_api_key = "xxx"

# Paramétrage de l'appel
id_ligne_idfm = 'C01742'; #RER A
api_idfm_url = f"https://prim.iledefrance-mobilites.fr/marketplace/general-message?LineRef=STIF%3ALine%3A%3A{id_ligne_idfm}%3A"
headers = {
    "apiKey": api_idfm_api_key
}

# Effectuer une requête GET avec l'en-tête contenant la clé d'API IDFM
response = requests.get(api_idfm_url, headers=headers)

# Extraction de la réponse au format json
try:
    if response.status_code == 200:
        # Extraire le JSON de la réponse
        json_data = response.json()
        # Vous pouvez afficher le JSON pour le vérifier
        print(json_data)
    else:
        print("Échec de la requête HTTP. Statut de réponse :", response.status_code)
except Exception as e:
    print("Une erreur s'est produite :", str(e))

### Onyxia : Lecture d'un fichier sur l'espace de stockage partagé (voir aussi notebook dédié)

In [ ]:
# Installation des dépendances :
!pip install s3fs

# import des dépendances
import os
import s3fs

# Creation d'un objet filesystem, les variables d'environnement on été automatiquement traitées par Onyxia
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

# Liste des documents du répertoire partagé dédié au hackathon :
fs.ls("dlb-hackathon/datasets-diffusion", refresh=True)

# Lecture d'un fichier csv et chargement en DataFrame Panda : 
import pandas as pd

BUCKET = "dlb-hackathon"
FILE_KEY_S3 = "datasets-diffusion/ascenseurs_historique_etat/RELEVES_ETATS_ASCENSEURS_SNCF_RATP.parquet"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    df = pd.read_parquet(file_in)

### Onyxia : Ecriture d'un fichier sur l'espace de stockage partagé (voir aussi notebook dédié)

In [ ]:
# On peut uploader par une commande terminal
!mc cp ./fichier_exemple.parquet s3/dlb-deptdata/SHARED_DATA

### Transformer un code ligne en un identifiant de ligne SIRI pour les appels d'API

In [ ]:
# Conversion de l'id_ligne du référentiel des lignes en id compatible avec les API, les "deux points" finaux sont indispensables
code_ligne_idfm = 'C01742'; #RER A
id_ligne_idfm = "STIF:Line::{id_ligne_idfm}:"

### Transformer un code arrêt en un identifiant d'arrêt SIRI pour les appels d'API. 
Note : les identifiant d'arrêts à prendre en compte sont en général les id de zones d'arrêt.

In [ ]:
# Conversion de ZdAId du référentiel des arrêts en id compatible avec les API, les "deux points" finaux sont indispensables
code_zone_arret_idfm = '42135'; #Les Dix Arpents
id_arret_idfm = "STIF:monomodalStopPlace::{code_zone_arret_idfm }:"

### Utilisation de modèles ouverts sur Hugging Face en local

In [ ]:
# installation de la version python locale d'ollama (llama-cpp)
!pip install llama-cpp-python

# installation du client hugging face (Le Github de l'IA)
!pip install -U "huggingface_hub[cli]"

# Chargement de llama-cpp
from llama_cpp import Llama
# Cette instruction minimale permet de sélectionner le modèle et son niveau de compression, le téléchargement le paramétrage et la mise en mémoire (CPU), sont automatiques :
llm = Llama.from_pretrained(
        repo_id="bartowski/gemma-2-2b-it-GGUF", #Plus gros dépôt de LLM libres de Hugging face : https://huggingface.co/bartowski
        filename="gemma-2-2b-it-Q6_K_L.gguf", # Attention a prendre des modèles de taille <= 6Go pour des performances raisonnables, pas de garantie de fonctionnement au delà 
 n_ctx=4096,
 verbose=False,
)

# Une fois le LLM chargé (environ une minute par Go), il peut être appelé, via une fonction *completion, ici pour un chatbot :
llm_response = llm.create_chat_completion(
        messages = [
                {
                        "role": "user",
                        "content": "###SYSTEM : You are the Capital assistant. I give you a country, you give me the capital and try to retrieve the country iso code. ### FORMAT : Capital=<name of the Capital> CODE=<ISO CODE 2 CHARS>
                },
                {
                        "role": "assistant",
                        "content": "OK, I understand. If you say France, i say Capital=Paris CODE=FR
                },
                {
                        "role": "user",
                        "content": "Germany"
                }
        ]
)

# La réponse du LLM est pleine de métadonnées, mais le message texte peut être extrait de cette manière simple :
print(llm_response['choices'][0]['message']['content'])
# Capital=Berlin CODE=DE

#Exemple avec un autre LLM :
llm2 = Llama.from_pretrained(
        repo_id="bartowski/Llama-3.2-3B-Instruct-GGUF", #Plus gros dépôt de LLM libres de Hugging face : https://huggingface.co/bartowski
        filename="Llama-3.2-3B-Instruct-Q5_K_M.gguf", # Attention a prendre des modèles de taille <= 6Go pour des performances raisonnables, pas de garantie de fonctionnement au delà 
 n_ctx=4096,
 verbose=False,
)
llm_response2 = llm2.create_chat_completion(
        messages = [
                {
                        "role": "user",
                        "content": "###SYSTEM : You are the Capital assistant. I give you a country, you give me the capital and try to retrieve the country iso code. ### FORMAT : Capital=<name of the Capital> CODE=FR"
                },
                {
                        "role": "assistant",
                        "content": "OK, I understand. If you say France, i say Capital=Paris CODE=FR"
                },
                {
                        "role": "user",
                        "content": "Germany"
                }
        ]
)
print(llm_response2['choices'][0]['message']['content'])
# Capital=Cologne CODE=DE ## no comment :)